In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ConfSelect import weighted_BH, weighted_CS, eval_sel
q=0.5
quantile = 0.9

seed = 8

# Step 1: Generate samples
np.random.seed(seed)  # For reproducibility
X1 = np.random.normal(0, 0.3, 16000)
X2 = np.random.normal(0.5, 0.5, 1000)

# Step 2: Combine X1 and X2, generate Y
X = np.concatenate((X1, X2))
e = np.random.normal(0, 0.3, X.shape[0])
Y = - X + X**3 + e

# Preparing the data for regression (adding X^2 and X^3)
X_poly = np.vstack([X, X**2, X**3]).T

# Assuming you have a logic for assigning weights
# Here's an example of creating weights (this should be adapted to your actual use case)
# For simplicity, we'll create weights inversely proportional to the standard deviation of each segment
# This is a placeholder and should be replaced with your specific weighting logic
weights = np.exp((X_poly[:, 0]+0.28)**2/(2*0.375*0.375))
#weights = weights / np.sum(weights)

# Add a constant to the model
X_poly_with_const = sm.add_constant(X_poly)

# Split your data to match the original OLS setup
X_train = X_poly_with_const[:8000]
Y_train = Y[:8000]
weights_train = weights[:8000]/np.sum(weights[:8000])

# Fit the WLS model
model = sm.WLS(Y_train, X_train, weights=weights_train).fit()

# Print the summary of the model
print(model.summary())


                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.317
Model:                            WLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     1235.
Date:                Tue, 19 Mar 2024   Prob (F-statistic):               0.00
Time:                        14:08:50   Log-Likelihood:                -3979.4
No. Observations:                8000   AIC:                             7967.
Df Residuals:                    7996   BIC:                             7995.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0137      0.005      2.519      0.0

In [2]:
dcalib = X_poly_with_const[8000:16000]
dtest = X_poly_with_const[16000:]

dother = np.concatenate((dcalib,dtest))
all_pred = model.predict(dother)
train_pred = model.predict(X_train)


hat_mu_calib = np.array(model.predict(dcalib))
hat_mu_test = np.array(model.predict(dtest))
y_calib = Y[8000:16000]
w_calib = weights[8000:16000]
y_test = Y[16000:]
w_test = weights[16000:]

In [3]:
# Calculate MSE
mse = np.mean((y_test - hat_mu_test) ** 2)
print(f"Mean Squared Error: {mse}")

# Calculate R^2 Score
y_mean = np.mean(y_test)
ss_tot = np.sum((y_test - y_mean) ** 2)
ss_res = np.sum((y_test - hat_mu_test) ** 2)
r2 = 1 - (ss_res / ss_tot)
print(f"R^2 Score: {r2}")

Mean Squared Error: 0.1261641407465864
R^2 Score: 0.7503247419593937


In [4]:
y_train = Y[:8000]


#c = 0
c = np.quantile(y_train, quantile) 

 
calib_scores_res = y_calib - hat_mu_calib
calib_scores_sub = - hat_mu_calib 
calib_scores_clip = 100 * (y_calib > c) + c * (y_calib <= c) - hat_mu_calib
 
test_scores = c - hat_mu_test

 
# ========================= 
# ## weighted BH procedure
# ========================= 

# use scores res, sub, and clip
BH_res = weighted_BH(calib_scores_res, w_calib, test_scores, w_test, q)  
BH_sub = weighted_BH(calib_scores_sub[y_calib <= c], w_calib[y_calib<=c], test_scores, w_test, q) 
BH_clip = weighted_BH(calib_scores_clip, w_calib, test_scores, w_test, q)

In [5]:
# =============================================================================
# # summarize FDP, power and selection sizes
# =============================================================================


#BH_res_fdp, BH_res_power = eval_sel(BH_res, y_test, np.array([c]*len(y_test)))
#BH_sub_fdp, BH_sub_power = eval_sel(BH_sub, y_test, np.array([c]*len(y_test)))
#BH_clip_fdp, BH_clip_power = eval_sel(BH_clip, y_test, np.array([c]*len(y_test))) 

# Assuming BH_res[0] contains the integer indices you want to use for evaluation
BH_res_indices = BH_res[0]
# Now pass these indices to the eval_sel function
BH_res_fdp, BH_res_power = eval_sel(BH_res_indices, y_test, np.array([c]*len(y_test)))

# Assuming BH_sub[0] contains the integer indices for the "sub" case
BH_sub_indices = BH_sub[0]
# Now pass these indices to the eval_sel function
BH_sub_fdp, BH_sub_power = eval_sel(BH_sub_indices, y_test, np.array([c]*len(y_test)))

# Assuming BH_clip[0] contains the integer indices for the "clip" case
BH_clip_indices = BH_clip[0]
# Now pass these indices to the eval_sel function
BH_clip_fdp, BH_clip_power = eval_sel(BH_clip_indices, y_test, np.array([c]*len(y_test)))


# Organize BH results for DataFrame
fdp = [BH_res_fdp, BH_sub_fdp, BH_clip_fdp]
power = [BH_res_power, BH_sub_power, BH_clip_power]
nsel = [len(BH_res_indices), len(BH_sub_indices), len(BH_clip_indices)]
ndiff = [0] * 3  # Assuming no difference for BH-only results
nsame = [len(BH_res_indices), len(BH_sub_indices), len(BH_clip_indices)]  # Assuming all selections are the same for BH-only results

# Create DataFrame for BH-only results
res_BH_only = pd.DataFrame({
    "FDP": fdp,
    "power": power,
    "nsel": nsel,
    "ndiff": ndiff,
    "nsame": nsame,
    "score": ["res", "sub", "clip"],
    "method": ["WBH"] * 3,
    "q": q,
    "seed": seed
})

# If you want to print or use res_BH_only DataFrame
print(res_BH_only)

   FDP  power  nsel  ndiff  nsame score method    q  seed
0    0      0     0      0      0   res    WBH  0.5     8
1    0      0     0      0      0   sub    WBH  0.5     8
2    0      0     0      0      0  clip    WBH  0.5     8
